In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wEpQ1jGKqYR6X5pwfmfQW2gnAWckRCpX9rZKfN9zxHj4L3MgF64Fno
Mounted at /gdrive


In [ ]:
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir
from datetime import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
# Input
root_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe'
# ActiGraph_60s_epoch = join(root_folder, 'ActiGraph_60s_epoch')
activPAL_15s_epoch = join(root_folder, 'activPAL_15s_epoch')

# sleep time input_data
sleep_time_file = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Rashmika_analysis/metadata/sleep_time.xlsx'

# Output
ActiPAL_output_folder = join(root_folder, '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/Data/processed_data/ActiPAL/epoch_data/15s_ref')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load Sleep Data

In [ ]:
sleep_df = pd.read_excel(sleep_time_file)

In [ ]:
sleep_df.head()

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap,age_yr,sex,BMI,bodymass_kg,AG_data,AP_data,"Side_TKA (1=L,2=R)",slnumnew,nonwearmins,sleepmins,nonwearpct,justnonwear,justsleep,av_inbedtime,av_outofbed_time
0,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52.0,female,76.86261,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22:00:00,06:00:00
1,1,HOA10,2019-06-24 13:59:47,2019-07-01 10:06:44,164h 6m 57d,2019-06-24 14:00:00,2019-07-01 07:59:59,161h 59m 59d,161h 59m 59d,51.0,female,42.41690,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21:00:00,09:00:00
2,2,HOA11,2019-07-01 13:59:47,2019-07-08 11:09:17,165h 9m 30d,2019-07-01 14:00:00,2019-07-08 07:59:59,161h 59m 59d,161h 59m 59d,56.0,male,30.09143,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:00:00,06:00:00
3,3,HOA12,2019-07-01 15:59:47,2019-07-08 10:44:11,162h 44m 24d,2019-07-01 16:00:00,2019-07-08 07:59:59,159h 59m 59d,159h 59m 59d,61.0,female,27.43484,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:00:00,05:00:00
4,4,HOA13,2019-07-08 16:59:47,2019-07-15 10:12:35,161h 12m 48d,NaT,NaT,NaN,NaN,56.0,female,36.73095,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:00:00,05:00:00


### MET from Activity Score


In [ ]:

def get_met_from_activity_counts_1min(score_df,step_count_df):
  output_df = (score_df-(4*step_count_df*4*15/120))/(15*(1-(step_count_df*4/120)))
  return output_df

def get_met_from_activity_counts_15s(score_df,step_count_df):
  output_df = (score_df-(4*step_count_df*15/120))/(15*(1-(step_count_df/120)))
  return output_df

def get_met_from_first_part(score_df):
  output_df = score_df/15
  return output_df

def get_met_from_step_count(row):
  if (thigh_epoch_data['Step Count'][row.name]==0):
    met_value = 1.25
  else:
    met_value = ((1.4*0.0167)+(4-1.4)*(thigh_epoch_data['Step Count'][row.name]*4/120)*0.0167)*60
  return met_value

In [ ]:
input_data = pd.DataFrame()
input_data['score'] = [22.2,22.73,18.75,18.75,18.75,18.75,18.75,18.75]
input_data['step'] = [2,2,0,0,0,0,0,0]

print(get_met_from_first_part(input_data['score']))
print(get_met_from_activity_counts_15s(input_data['score'],input_data['step']))
print(get_met_from_activity_counts_1min(input_data['score'],input_data['step']))

0    1.480000
1    1.515333
2    1.250000
3    1.250000
4    1.250000
5    1.250000
6    1.250000
7    1.250000
Name: score, dtype: float64
0    1.437288
1    1.473220
2    1.250000
3    1.250000
4    1.250000
5    1.250000
6    1.250000
7    1.250000
dtype: float64
0    1.300000
1    1.337857
2    1.250000
3    1.250000
4    1.250000
5    1.250000
6    1.250000
7    1.250000
dtype: float64


### Process ref values

In [ ]:
epoch_files = [f for f in listdir(activPAL_15s_epoch) if (isfile(join(activPAL_15s_epoch, f)) and ('Events' not in f))]

print(epoch_files)

['HOA1-AP479873 06May19 03-00pm for 6d 17h 49m-CREA-PB08100943-15sEpochs.csv', 'HOA10-AP479868 24Jun19 02-00pm for 6d 20h 6m-CREA-PB08100943-15sEpochs.csv', 'HOA11-AP472608 01Jul19 02-00pm for 6d 21h 9m-CREA-PB08100943-15sEpochs.csv', 'HOA12-AP472598 01Jul19 04-00pm for 6d 18h 44m-CREA-PB08100943-15sEpochs.csv', 'HOA13-AP472593 08Jul19 05-00pm for 6d 17h 12m-CREA-PB08100943-15sEpochs.csv', 'HOA14-AP472583 15Jul19 03-00pm for 6d 19h 17m-CREA-PB08100943-15sEpochs.csv', 'HOA15-AP472607 29Jul19 03-00pm for 6d 19h 53m-CREA-PB08100943-15sEpochs.csv', 'HOA16-AP472598 29Jul19 04-00pm for 6d 18h 39m-CREA-PB08100943-15sEpochs.csv', 'HOA17-AP472593 29Jul19 04-00pm for 6d 18h 45m-CREA-PB08100943-15sEpochs.csv', 'HOA18-AP472604 29Jul19 05-00pm for 1d 2h 22m-CREA-PB08100943-15sEpochs.csv', 'HOA19-AP472607 202a 5Aug19 6-00pm for 6d 17h 21m-CREA-PB08100943-15sEpochs.csv', 'HOA2-AP472595 06May19 03-00pm for 7d 0m-CREA-PB08100943-15sEpochs.csv', 'HOA20-AP472607 202a 19Aug19 5-00pm for 6d 17h 19m-CREA-PB

In [ ]:
epoch_files[156]

'OA76-AP472596 17Oct16 03-00pm for 7d 0m-CREA-PB08100943-15sEpochs.csv'

In [ ]:
#check#OA46
# oafile = join(ActiGraph_60s_epoch, epoch_files[156])
# print(oafile)
# oa46 = pd.read_csv(oafile, skiprows=11)
# oa46.head()

sleep_df.loc[sleep_df['Participant'] == 'OA46']

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap,age_yr,sex,BMI,bodymass_kg,AG_data,AP_data,"Side_TKA (1=L,2=R)",slnumnew,nonwearmins,sleepmins,nonwearpct,justnonwear,justsleep,av_inbedtime,av_outofbed_time
500,143,OA46,2016-03-08 15:59:48,2016-03-14 12:14:22,140h 14m 34d,NaT,NaT,NaN,NaN,59.0,female,39.3,85.0,NaN,NaN,2.0,1.0,0.0,333.0,0.0,NaN,1.0,2016-03-07 21:45:00,2016-03-08 03:17:00
501,143,OA46,2016-03-08 15:59:48,2016-03-14 12:14:22,140h 14m 34d,NaT,NaT,NaN,NaN,59.0,female,39.3,85.0,NaN,NaN,2.0,2.0,0.0,212.0,0.0,NaN,1.0,2016-03-09 01:39:00,2016-03-09 05:10:00
502,143,OA46,2016-03-08 15:59:48,2016-03-14 12:14:22,140h 14m 34d,NaT,NaT,NaN,NaN,59.0,female,39.3,85.0,NaN,NaN,2.0,3.0,0.0,188.0,0.0,NaN,1.0,2016-03-09 21:58:00,2016-03-10 01:05:00
503,143,OA46,2016-03-08 15:59:48,2016-03-14 12:14:22,140h 14m 34d,NaT,NaT,NaN,NaN,59.0,female,39.3,85.0,NaN,NaN,2.0,4.0,0.0,377.0,0.0,NaN,1.0,2016-03-12 01:03:00,2016-03-12 07:19:00
504,143,OA46,2016-03-08 15:59:48,2016-03-14 12:14:22,140h 14m 34d,NaT,NaT,NaN,NaN,59.0,female,39.3,85.0,NaN,NaN,2.0,5.0,0.0,486.0,0.0,NaN,1.0,2016-03-12 23:00:00,2016-03-13 07:05:00


In [ ]:
for f in tqdm(epoch_files):

  user_id = f.split('-')[0]

  thigh_epoch_data = pd.read_csv(join(activPAL_15s_epoch, f), skiprows=11, usecols=[1,2], header=None)
  thigh_epoch_data.columns = ['Step Count', 'Activity_score']

  # Create time column
  print(user_id)
  if(user_id=='OA13' or user_id=='OA20'):continue
  start_time = sleep_df.loc[sleep_df['Participant'] == user_id].iloc[0]['activpal_Start_time']
  # if user_id != 'OA46_fixed60sec.csv':
  #   start_time = sleep_df.loc[sleep_df['Participant'] == user_id].iloc[0]['actigraph_Start_time']
  # else:
  #   start_time = "07-03-2016  4:00:00"

  thigh_epoch_data['Time_stamp'] = list(pd.date_range(start_time, periods=thigh_epoch_data.shape[0], freq='15S'))
  # print(hip_epoch_data['Time_stamp'])
  # break

  # thigh_epoch_data['thigh_EE'] = get_met_from_activity_counts_15s(thigh_epoch_data['Activity_score'],thigh_epoch_data['Step Count'])
  thigh_epoch_data['thigh_EE'] = thigh_epoch_data.apply(get_met_from_step_count,axis=1)
  # print(thigh_epoch_data)
  # break
  # hip_epoch_data['waist_vm_60'] = np.sqrt([(hip_epoch_data.AxisX ** 2) + (hip_epoch_data.AxisY ** 2) + (hip_epoch_data.AxisZ ** 2)])[0]
  # hip_epoch_data['waist_vm_cpm'] = hip_epoch_data['waist_vm_60']  # cpm = counts per minute
  # hip_epoch_data['waist_cpm'] = hip_epoch_data.AxisY

  # hip_epoch_data['waist_ee'] = hip_epoch_data.apply(get_freedson_vm3_combination_11_energy_expenditure, axis=1)

  # # Use following snippet if need the reference data for granular time_epochs
  # multiplication_factor = int(60 / time_epoch)
  # epoch_hip_epoch_data = epoch_hip_epoch_data.reindex(np.repeat(epoch_hip_epoch_data.index.values, multiplication_factor), method='ffill')

  # save output file
  # Input example: 'HOA1 (2019-05-06)60sec.csv'
  output_filename = f.replace('.csv', '_15sREF.csv')
  thigh_epoch_data.to_csv(join(ActiPAL_output_folder, output_filename), sep=',', index=None)
  del thigh_epoch_data


  0%|          | 0/167 [00:00<?, ?it/s]

HOA1


  1%|          | 1/167 [00:03<11:02,  3.99s/it]

HOA10


  1%|          | 2/167 [00:05<09:03,  3.30s/it]

...

 99%|█████████▉| 165/167 [04:58<00:03,  1.94s/it]

OA89


 99%|█████████▉| 166/167 [05:00<00:01,  1.94s/it]

OA9


100%|██████████| 167/167 [05:02<00:00,  1.81s/it]
